**Natural Language Inferencing (NLI)**

Natural language Inferencing is a classic NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis), and deciding how they are related - If the premise entails the hypothesis, contradicts it, or neither

Dataset - Contradictory, My Dear Watson

Tools - Tensorflow 2, Keras and BERT

In [ ]:
import numpy as np #linear algebra
import pandas as pd # data processing

import os
for dirname, _ , filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [ ]:
# Setting up TPU

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# Downloading Data
# The training set contains a premise, a hypothesis, a label (0 = entailment, 1 = neutral, 2 = contradiction), and the language of the text.

train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

In [ ]:
train.head()

In [ ]:
train.premise.values[1]

In [ ]:
train.hypothesis.values[1]

In [ ]:
train.label.values[1]

In [ ]:
# look at the distribution of languages in the training set.

labels, frequencies = np.unique(train.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies, labels = labels, autopct = "%1.1f%%")
plt.show()

In [ ]:
# Preparing Data for Input
# To start out, we can use a pretrained model. Here, we'll use a multilingual
# BERT model from huggingface. 
# More info at: https://github.com/google-research/bert/blob/master/multilingual.md

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# Tokenizers turn sequences of words into arrays of numbers. Let's look at an example:

def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
encode_sentence("I love natural language processing")

BERT uses three kind of input data - input word IDs, input masks and input typoe IDs.

These allow the model to know that the premise and hypothesis are distinct sentences and also to ignore any padding from the tokenizer.

We add a [CLS] token to denote the beginning of the inputs, and a [SEP] token to denote the separation between the premise and the hypothesis. We also need to pad all of the inputs to be the same size. For more information about BERT inputs, see: https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel

Now, we're going to encode all of our premise/hypothesis pairs for input into BERT.

In [ ]:
def bert_encode(hypotheses, premises, tokenzier):
    
    num_examples = len(hypotheses)
    
    sentence1 = tf.ragged.constant([encode_sentence(s) for s in np.array(hypotheses)])
    sentence2 = tf.ragged.constant([encode_sentence(s) for s in np.array(premises)])
    
    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)
    
    input_mask = tf.ones_like(input_word_ids).to_tensor()
    
    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)

    input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor()
    
    inputs = {'input_word_ids' : input_word_ids.to_tensor(),
             'input_mask': input_mask,
             'input_type_ids': input_type_ids}
    
    return inputs

In [ ]:
train_input = bert_encode(train.premise.values, train.hypothesis.values, tokenizer)

**Creating & Training Model**

Now, we can incorporate the BERT transformer into a Keras Functional Model. For more information about the Keras Functional API, see: https://www.tensorflow.org/guide/keras/functional.

This model was inspired by the model in this notebook: https://www.kaggle.com/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert#BERT-and-Its-Implementation-on-this-Competition, which is a wonderful introduction to NLP!

In [ ]:
max_len = 50

def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
with strategy.scope():
    model = build_model()
    model.summary()

In [ ]:
model.fit(train_input, train.label.values, epochs=5, verbose=1, batch_size=64, validation_split=0.2)

In [ ]:
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
test_input = bert_encode(test.premise.values, test.hypothesis.values, tokenizer)

In [ ]:
test.head()

**Generating & Submitting Predictions**

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [ ]:
submission.head()

In [ ]:
# submission file generated for the competition
submission.to_csv("submission.csv", index = False)